<a href="https://colab.research.google.com/github/Nerflix/Split_Horizon/blob/main/Split_Horizon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3

In [1]:
class Router:
    def __init__(self, name):
        # Initialize router attributes
        self.name = name
        self.distance_table = {}
        self.routing_table = {}

    def update_distance_table(self, destination, next_hop, distance):
        # Update distance table
        self.distance_table[destination] = (next_hop, distance)

    def update_routing_table(self):
        # Update routing table based on distance table
        for destination, (next_hop, _) in self.distance_table.items():
            self.routing_table[destination] = (next_hop, self.distance_table[destination][1])

    def print_distance_table(self, step):
        # Print distance table
        print(f"{self.name} Distance Table at t={step}")
        print("    ", end="")
        for destination in sorted(self.distance_table.keys()):
            print(f"{destination:<5}", end="")
        print()
        for router in sorted(routers, key=lambda r: r.name):
            print(f"{router.name}   ", end="")
            for destination in sorted(self.distance_table.keys()):
                next_hop, distance = self.distance_table[destination]
                if router.name == next_hop:
                    print(f"{distance:<5}", end="")
                else:
                    print("INF  ", end="")
            print()
        print()

    def print_routing_table(self):
        # Print routing table
        print(f"{self.name} Routing Table:")
        for destination, (next_hop, distance) in sorted(self.routing_table.items()):
            print(f"{destination},{next_hop},{distance}")
        print()

In [2]:
def initialize_topology():
    # Create routers and return the list
    routers = []
    while True:
        router_name = input().strip()
        if not router_name.isalpha() or not router_name.isupper():
            print("Error: Invalid input.Router names must be uppercase letters and Spelled Correctly.")
            continue
        if router_name == "DISTANCEVECTOR":
            break
        if router_name == "UPDATE":
            print("Error: Invalid input. The name UPDATE must be entered after all links have been entered.")
            continue
        router = Router(router_name)
        routers.append(router)
    return routers

In [3]:
def initialize_links(routers):
    # Set up links between routers
    while True:
        link = input().strip()
        # When input is "UPDATE"
        if link == "UPDATE":
            break
        if link == "DISTANCEVECTOR":
            print("Error: Invalid input. The name DISTANCEVECTOR must be entered after all router names have been entered.")
            continue
        # Check if link is in the expected format
        link_parts = link.split()
        if len(link_parts) != 3:
            print("Error: Invalid input. Link information must be in the format: source destination weight chech the values spacing and spelling")
            continue
        # Read info from input
        source, destination, weight = link_parts
        if not source.isalpha() or not source.isupper() or not destination.isalpha() or not destination.isupper():
            print("Error: Invalid input. Router names must be uppercase letters.")
            continue
        weight = int(weight)
        source_router = next(r for r in routers if r.name == source)
        # Update distance tables
        destination_router = next(r for r in routers if r.name == destination)
        source_router.update_distance_table(destination, destination, weight)
        destination_router.update_distance_table(source, source, weight)

In [4]:
def update_topology(routers):
    # Update network topology
    while True:
        update = input().strip()
        # When the input is "END"
        if update == "END":
            break
        # Check if input is in the expected format
        if len(update.split()) != 3:
            print("Error: Invalid input. Expected format is upercase: source destination weight")
            continue
        # Read input info
        source, destination, weight = update.split()
        if not source.isalpha() or not source.isupper() or not destination.isalpha() or not destination.isupper():
            print("Error: Invalid input. Router names must be uppercase letters.")
            continue
        weight = int(weight)
        # Find routers by names
        source_router = next((r for r in routers if r.name == source), None)
        destination_router = next((r for r in routers if r.name == destination), None)
        # If weight is -1, INF
        if weight == -1:
            if source_router and destination in source_router.distance_table:
                source_router.update_distance_table(destination, destination, float('inf'))
            if destination_router and source in destination_router.distance_table:
                destination_router.update_distance_table(source, source, float('inf'))
        else:
            # If weight is not -1, create new router instances
            if source_router is None:
                source_router = Router(source)
                routers.append(source_router)
            if destination_router is None:
                destination_router = Router(destination)
                routers.append(destination_router)
            # Update distance tables
            source_router.update_distance_table(destination, destination, weight)
            destination_router.update_distance_table(source, source, weight)


In [5]:
def distance_vector(routers):
    # Perform distance vector routing algorithm with split horizon
    converged = False
    step = 0
    while not converged:
        converged = True
        for router in routers:
            # Iterate over the items in the distance table of the router
            for destination, (next_hop, distance) in list(router.distance_table.items()):
                # Iterate over the neighbor routers
                for neighbor in routers:
                    # Check if the destination is in the neighbor's distance table and at the meantime neighbor is not next hop
                    if destination in neighbor.distance_table and neighbor.name != next_hop:
                        # Split horizon: do not advertise a route back to the router from which it was learned
                        if neighbor.distance_table[destination][0] == router.name:
                            continue
                        # Calculate the new distance
                        new_distance = distance + neighbor.distance_table[destination][1]
                        # Update distance table
                        if destination not in router.distance_table or new_distance < router.distance_table[destination][1]:
                            router.update_distance_table(destination, neighbor.name, new_distance)
                            converged = False
        # Update routing table and print
        for router in routers:
            router.update_routing_table()
            router.print_distance_table(step)
        step += 1
    for router in routers:
        router.print_routing_table()
    print("Convergence has been reached.")


In [6]:
if __name__ == "__main__":
    # Initialize topology
    routers = initialize_topology()

    # Initialize links
    initialize_links(routers)

    # Perform distance vector algorithm
    distance_vector(routers)

    # Update topology
    update_topology(routers)

    # Perform distance vector algorithm again
    distance_vector(routers)


X
Y
Z
DISTANCEVECTOR
X Z 8
X Y 2
Y Z 3
UPDATE
X Distance Table at t=0
    Y    Z    
X   INF  INF  
Y   2    INF  
Z   INF  8    

Y Distance Table at t=0
    X    Z    
X   2    INF  
Y   INF  INF  
Z   INF  3    

Z Distance Table at t=0
    X    Y    
X   8    INF  
Y   INF  3    
Z   INF  INF  

X Routing Table:
Y,Y,2
Z,Z,8

Y Routing Table:
X,X,2
Z,Z,3

Z Routing Table:
X,X,8
Y,Y,3

Convergence has been reached.
Y Z -1
X Z 3
END
X Distance Table at t=0
    Y    Z    
X   INF  INF  
Y   2    INF  
Z   INF  3    

Y Distance Table at t=0
    X    Z    
X   2    INF  
Y   INF  INF  
Z   INF  inf  

Z Distance Table at t=0
    X    Y    
X   3    INF  
Y   INF  inf  
Z   INF  INF  

X Routing Table:
Y,Y,2
Z,Z,3

Y Routing Table:
X,X,2
Z,Z,inf

Z Routing Table:
X,X,3
Y,Y,inf

Convergence has been reached.
